# 作业1.1

## 1 利用时间和能量信息，计算粒子入射位置tx和qx

In [1]:
%jsroot on
TH1D *tdiff = new TH1D("tdiff","td-tu",140,-20,50);
TCanvas *c1 = new TCanvas("c1","c1");

Double_t tu,td,ctof,tof,x,e,qu,qd;
int pid;

TFile *ipf = new TFile("tree.root");
TTree *tree = (TTree*)ipf->Get("tree");

tree->SetBranchAddress("tu",&tu);
tree->SetBranchAddress("td",&td);
tree->SetBranchAddress("ctof",&ctof);
tree->SetBranchAddress("tof",&tof);
tree->SetBranchAddress("x",&x);
tree->SetBranchAddress("e",&e);
tree->SetBranchAddress("qu",&qu);
tree->SetBranchAddress("qd",&qd);
tree->SetBranchAddress("pid",&pid);

In [2]:
Long64_t nentries=tree->GetEntries();//得到事件总数
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    tdiff->Fill(td-tu);  // if(ng==1) tx->Fill(tu-td), 只写入满足给定条件的事件      
  }
tdiff->Draw();
c1->Draw();

In [3]:
TH1D *dtd=new TH1D("dtd","dt/dx",141,-20.25,50.25);
for(int i=1;i<tdiff->GetNbinsX();i++) {
    Double_t df=tdiff->GetBinContent(i+1)-tdiff->GetBinContent(i);
    dtd->Fill(tdiff->GetBinLowEdge(i+1),df);
}
dtd->Sumw2(0);
dtd->Draw();
dtd->Fit("gaus","","",-14,-10.5);//txl
c1->Draw();

 FCN=156.822 FROM MIGRAD    STATUS=CONVERGED      70 CALLS          71 TOTAL
                     EDM=3.63466e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.84345e+02   1.21045e+01   6.31804e-02   3.70980e-07
   2  Mean        -1.18782e+01   2.23495e-02   1.36380e-04  -4.33732e-04
   3  Sigma        5.53250e-01   1.51037e-02   4.61133e-05   3.13404e-03


In [4]:
TF1 *f1 = new TF1("f1","[0]*TMath::Exp(-0.5*((x-[1])/[2])^2)",40,43.5);

In [5]:
//c1->SetLogy(0);
f1->SetParameter(0,-350);
f1->SetParameter(1,41.5);
f1->SetParameter(2,0.5);
dtd->Fit("f1","R");
dtd->Draw();
c1->Draw();

 FCN=23.7676 FROM MIGRAD    STATUS=CONVERGED      73 CALLS          74 TOTAL
                     EDM=6.81548e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -2.64129e+02   1.08472e+01   2.09064e-02  -1.89513e-05
   2  p1           4.15012e+01   2.66764e-02   5.83476e-05  -4.41855e-02
   3  p2           7.16592e-01   1.91389e-02   3.43322e-05   3.37612e-03


## time-position
* ### $ t_x = t_d - t_u $
* ### $ t_{xl} = -11.8782, t_{xr} = 41.5012 $
* ### $ t_{xoff} = (t_{xl} + t_{xr})/2 = 14.8115 $ 
    ### $$ x = \frac{2L}{t_{xr}-t_{xl}}*(t_x-t{xoff})= 3.7468*(t_x-14.8115) $$

In [6]:
TH1D *htx=new TH1D("htx","htx",500,-120,120);
TH2D *hdx=new TH2D("hdx","htx-hx:hx",100,-20,20,500,-120,120);
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t tx=3.7468*(td-tu-14.8115);
    htx->Fill(tx);
    hdx->Fill(tx-x,x);//difference
  }
htx->Draw();
c1->Draw();

## residual graph

In [7]:
hdx->Draw("colz");
c1->Draw();

In [8]:
TH1D *hdx1=hdx->ProjectionX("projx of hdx");
hdx1->Draw();
hdx1->Fit("gaus");//中心值 mean x=0.55, 说明确定td-tu的边界时可能有些问题。[3],[6]
c1->Draw();

 FCN=57.6443 FROM MIGRAD    STATUS=CONVERGED      52 CALLS          53 TOTAL
                     EDM=5.70037e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     7.07131e+03   2.73595e+01   8.36330e-02  -4.19783e-06
   2  Mean         3.28029e-01   7.13746e-03   2.66861e-05  -1.97218e-03
   3  Sigma        2.25547e+00   5.02860e-03   2.26978e-06  -1.55059e-01


In [9]:
TH1D *qdiff = new TH1D("qdiff","log(qu/qd)",100,-1,1);

//Long64_t nentries=tree->GetEntries();//得到事件总数
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    qdiff->Fill(TMath::Log(qu/qd));  // if(ng==1) tx->Fill(tu-td), 只写入满足给定条件的事件      
  }
qdiff->Draw();
c1->Draw();

In [10]:
TH1D *dqd=new TH1D("dqd","dq/dx",101,-1.01,1.01);
for(int i=1;i<qdiff->GetNbinsX();i++) {
    Double_t df=qdiff->GetBinContent(i+1)-qdiff->GetBinContent(i);
    dqd->Fill(qdiff->GetBinLowEdge(i+1),df);
}


In [11]:
dqd->Sumw2(0);
dqd->Draw();
dqd->Fit("gaus","","",-0.65,-0.45);
c1->Draw();

 FCN=106.844 FROM MIGRAD    STATUS=CONVERGED      66 CALLS          67 TOTAL
                     EDM=1.63947e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     2.72662e+02   9.49339e+00   3.55776e-02   2.69019e-07
   2  Mean        -5.38141e-01   1.36799e-03   6.55149e-06  -2.09848e-02
   3  Sigma        4.56845e-02   1.26906e-03   3.47739e-05   4.50736e-03


In [12]:
TF1 *f2 = new TF1("f2","[0]*TMath::Exp(-0.5*((x-[1])/[2])^2)",0.48,0.7);

In [13]:
c1->SetLogy(0);
f2->SetParameter(0,-350);
f2->SetParameter(1,0.5);
f2->SetParameter(2,0.04);
dqd->Fit("f2","R");
dqd->Draw();
c1->Draw();

 FCN=151.088 FROM MIGRAD    STATUS=CONVERGED      95 CALLS          96 TOTAL
                     EDM=3.28948e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -2.36193e+02   8.31922e+00   3.86092e-02   2.32154e-07
   2  p1           5.40595e-01   1.58806e-03   9.23768e-06   9.67522e-02
   3  p2           4.96255e-02   1.47106e-03   6.70051e-06  -1.21709e-01


## energy-position
* ### $ q_x = log(q_u/q_d) $
* ### $ q_{xl} = -0.5381, q_{xr} = 0.5406 $
* ### $ q_{xoff} = (q_{xl} + q_{xr})/2 = -0.0013 $ 
    ### $$ x = \frac{2L}{q_{xr}-q_{xl}}*(q_x-q{xoff})= 185.4084*(q_x+0.0013) $$

In [14]:
TH1D *hqx=new TH1D("hqx","hqx",500,-120,120);
TH2D *hdqx=new TH2D("hdqx","hqx-hx:hx",200,-100,100,500,-120,120);
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t qx=185.4084*(TMath::Log(qu/qd)+0.0013);
    hqx->Fill(qx);
    hdqx->Fill(qx-x,x);//difference
  }
hqx->Draw();
c1->Draw();

In [15]:
hdqx->Draw("colz");
c1->Draw();

In [16]:
TH1D *hdqx1=hdqx->ProjectionX("projx of hdqx");
hdqx1->Draw();
hdqx1->Fit("gaus");
c1->Draw();

 FCN=83.5871 FROM MIGRAD    STATUS=CONVERGED      58 CALLS          59 TOTAL
                     EDM=2.87503e-12    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.54015e+03   1.36796e+01   5.02949e-02   2.01302e-07
   2  Mean         2.13846e-01   3.56406e-02   1.60036e-04   4.55956e-06
   3  Sigma        1.12602e+01   2.50023e-02   2.71894e-06   3.22376e-03


# Conclusion
* ## tx is better than qx for the smaller sigma

# the method of absolutely calibration
* ### $ t_u=TOF+(L-x)/v_{sc}+T_{uoff} $
* ### $ t_d=TOF+(L+x)/v_{sc}+T_{doff} $
* ### $ TOF=(t_u+t_d)/2 - L/x_{sc} - (T_{uoff}-T_{doff})/2 $
* ### $ C=const,TOF=(t_u+t_d)/2+C $
* ### $ d_0=502.5cm,TOF_0=16.7483 $ $ ns $

In [17]:
TH2D *hgtofx=new TH2D("hgtofx","hgtofx",100,-120,120,100,39,48);
TH1D *hgctof=new TH1D("hgctof","hgctof",100,39,48);

In [18]:
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t tx=3.7468*(td-tu-14.8115);
    if(ctof>42&& ctof<44.5) { 
        hgtofx->Fill(tx,ctof);
        if(abs(tx)<5) hgctof->Fill(ctof);//gamma hits the center of the det.
    }
  }
hgtofx->Draw("colz");
c1->Draw();

In [19]:
hgctof->Draw();
hgctof->Fit("gaus");
c1->Draw();

 FCN=21.5743 FROM MIGRAD    STATUS=CONVERGED      54 CALLS          55 TOTAL
                     EDM=3.3555e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.84395e+02   5.98512e+00   1.08992e-02   2.54215e-07
   2  Mean         4.29561e+01   7.75199e-03   2.04831e-05   3.33618e-03
   3  Sigma        3.00497e-01   6.16077e-03   1.23267e-05   1.02535e-04


* ### $ TOF(0) =42.9561 $ $ ns $
* ### $ C=TOF_0-TOF(0)= -26.2078 $ $ns $

In [20]:
TH2D *hgtofcx=new TH2D("hgtofcx","corrected TOF",100,-120,120,100,2.9,3.75);
TH1D *htofc=new TH1D("htofc","htof",200,0,40);

In [21]:
for(Long64_t jentry=0; jentry<nentries; jentry++) {//对每个事件进行遍历
    tree->GetEntry(jentry);
    Double_t tx=3.7468*(td-tu-14.8115);
    Double_t d=TMath::Sqrt(502.5*502.5+tx*tx);
    Double_t tofc=(ctof-26.2078)/d*100.;//normalized to 500cm
    hgtofcx->Fill(tx,tofc);//gamma hits the center of the det.
    htofc->Fill(tofc);
  }
hgtofcx->Draw("colz");//tofc与x之间无关联
c1->Draw();

In [22]:
c1->SetLogy();
htofc->Draw();//修正后的飞行时间谱。
c1->Draw();

# calculation the ce and creat the new root file

In [23]:
Double_t ntof,ce;
TH1D *hce=new TH1D("hce","hce",300,-5,200);
TH2D *hcee=new TH2D("hcee","hce-he:he",100,-20,20,500,-20,200);

In [24]:
for (Long64_t jentry=0;jentry<nentries;jentry++)
{
    tree->GetEntry(jentry);
    Double_t tx=3.7468*(td-tu-14.8115);
    Double_t d=TMath::Sqrt(502.5*502.5+tx*tx);
    ntof=(ctof-26.2078)/d*100.;
    
    if(pid==1) 
    {
        ce = TMath::Sq(72/ntof);
    }
   // else ce=1;
    
    if(pid==1)
    {
        hce->Fill(ce);
        hcee->Fill(ce-e,e);
    }
}


In [30]:
hce->Fit("gaus");
hce->Draw();
c1->Draw();

 FCN=197.723 FROM MIGRAD    STATUS=CONVERGED      59 CALLS          60 TOTAL
                     EDM=1.68879e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     8.90069e+02   4.15812e+00   2.31838e-02   1.46956e-04
   2  Mean         1.00459e+02   8.11128e-02   5.58054e-04  -3.52700e-03
   3  Sigma        2.13931e+01   5.87785e-02   5.09438e-06   5.16196e-01


# the energy of nrutron:
* ### $ Energy_n=100.459 $ $ MeV $

In [26]:
c1->SetLogy(0);
hcee->Draw("colz");
c1->Draw();

In [28]:

Double_t tx,qx;
TFile *opf = new TFile("tree2.root","recreate");
TTree *opt = new TTree("mytree","tree");

opt->Branch("x",&x,"x/D");
opt->Branch("e",&e,"e/D");
opt->Branch("tof",&tof,"tof/D");
opt->Branch("ctof",&ctof,"cof/D");
opt->Branch("pid",&pid,"pid/I");
opt->Branch("tu",&tu,"tu/D");
opt->Branch("td",&td,"td/D");
opt->Branch("qu",&qu,"qu/D");
opt->Branch("qd",&qd,"qd/D");
opt->Branch("ntof",&ntof,"ntof/D");
opt->Branch("tx",&tx,"tx/D");
opt->Branch("tx",&qx,"qx/D");
opt->Branch("ce",&ce,"ce/D");


In [29]:

for (Long64_t jentry=0;jentry<nentries;jentry++)
{
    tree->GetEntry(jentry);
    tx=3.7468*(td-tu-14.8115);
    qx=185.4084*(TMath::Log(qu/qd)+0.0013);
    Double_t d=TMath::Sqrt(502.5*502.5+tx*tx);
    ntof=(ctof-26.2078)/d*100.;
    
    if(pid==1) ce = TMath::Sq(72/ntof);
    else ce=1;
    
    opt->Fill();   
}

opt->Write();
opf->Close();
